In [98]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [99]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [100]:
tf.keras.backend.clear_session()

In [101]:
subject = 'Acre - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [102]:
data = pd.read_csv('2003_mo_model_input_AC.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
# data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Acre - IDH Longevidade,Acre - IDH Renda,Acre - Desemprego,Acre - IDH,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado,Acre - IDH Educacao,Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),Contratação SFH (Bi R$),Contratação Jurídico Livre (Bi R$),Contratação Jurídico SFH (Bi R$),Direcionamento Aquisição (R$),Direcionamento Construção (R$),Direcionamento Aplicação (R$),Ativos Problemáticos - Home equity (R$),Ativos Problemáticos - SFH (R$),Ativos Problemáticos - FGTS (R$),Carteira de Crédito – Home equity (R$),Carteira de Crédito – Livre (R$),Carteira de Crédito - FGTS (R$),Carteira de Crédito - SFH (R$),Carteira de Crédito P.Jurídica - Comercial (R$),Carteira de Crédito P.Jurídica - Livre (R$),Carteira de Crédito P.Jurídica - FGTS (R$),Inadimplência - SFH (%),Inadimplência - FGTS (%),Inadimplência - Livre (%),Inadimplência - Comercial (%),Inadimplência - Home equity (%),Poupança e Empréstimo - SBPE (R$),Títulos - LH (R$),Títulos – LIG (R$),Títulos – CRI (R$),Títulos – LCI (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Acre - Consumo de Cimento (t)
0,2003-1,0.773065,0.651726,8.722906,0.691320,7.285707e+06,413409.255032,9.318417,6.587220e+06,0.643805,2.014625e+08,1.542506e+09,9.248956e+08,3.117876e+09,4.235269e+08,5.871672e+08,1.300935e+11,1.039573e+10,1.145754e+11,4.355660e+08,3.647348e+09,4.601248e+09,9.143676e+09,7.702509e+10,6.664510e+10,7.035792e+10,4.872928e+08,1.970044e+10,2.021690e+10,1.557727,2.468118,1.603214,0.300795,3.988206,1.540380e+11,1.211245e+09,8.107262e+09,1.921789e+10,8.202881e+10,0.724032,-5331.049150,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,3.260
1,2003-2,0.773454,0.651847,8.718028,0.691617,7.294881e+06,413607.843560,9.319990,6.590338e+06,0.644270,2.015196e+08,1.542506e+09,9.267621e+08,3.118866e+09,4.235269e+08,5.871672e+08,1.300935e+11,1.039573e+10,1.145754e+11,4.373259e+08,3.662680e+09,4.601248e+09,9.149335e+09,7.702509e+10,6.664510e+10,7.035792e+10,4.872928e+08,1.970044e+10,2.021690e+10,1.557727,2.467045,1.603118,0.300795,3.991229,1.540380e+11,1.211245e+09,8.107262e+09,1.921789e+10,8.222716e+10,0.690297,-5318.079644,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,2.191
2,2003-3,0.773843,0.651968,8.713149,0.691914,7.304055e+06,413806.432089,9.321563,6.593455e+06,0.644734,2.015529e+08,1.542506e+09,9.290952e+08,3.119443e+09,4.235269e+08,5.871672e+08,1.300935e+11,1.039573e+10,1.145754e+11,4.383525e+08,3.671623e+09,4.601248e+09,9.152636e+09,7.702509e+10,6.664510e+10,7.035792e+10,4.872928e+08,1.970044e+10,2.021690e+10,1.557727,2.466420,1.603063,0.300795,3.992993,1.540380e+11,1.211245e+09,8.107262e+09,1.921789e+10,8.234287e+10,0.669681,-5436.417870,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,2.154
3,2003-4,0.774233,0.652088,8.708271,0.692212,7.313229e+06,414005.020618,9.323136,6.596572e+06,0.645199,2.015565e+08,1.542506e+09,9.304019e+08,3.119505e+09,4.235269e+08,5.871672e+08,1.300935e+11,1.039573e+10,1.145754e+11,4.384625e+08,3.672581e+09,4.601248e+09,9.152990e+09,7.702509e+10,6.664510e+10,7.035792e+10,4.872928e+08,1.970044e+10,2.021690e+10,1.557727,2.466353,1.603057,0.300795,3.993182,1.540380e+11,1.211245e+09,8.107262e+09,1.921789e+10,8.235526e+10,0.660494,-5707.015274,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,2.643
4,2003-5,0.774622,0.652209,8.703393,0.692509,7.322403e+06,414203.609146,9.324709,6.599689e+06,0.645663,2.017543e+08,1.544532e+09,9.329889e+08,3.125213e+09,4.241975e+08,5.877836e+08,1.301077e+11,1.039463e+10,1.147057e+11,4.391591e+08,3.686861e+09,4.603092e+09,9.157499e+09,7.700499e+10,6.663242e+10,7.038333e+10,4.891177e+08,1.970361e+10,2.022233e+10,1.557311,2.465767,1.601718,0.301067,3.992907,1.543134e+11,1.211479e+09,8.089349e+09,1.921843e+10,8.245980e+10,0.648337,-5599

In [103]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,Acre - IDH Longevidade,Acre - IDH Renda,Acre - Desemprego,Acre - IDH,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado,Acre - IDH Educacao,Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),Contratação SFH (Bi R$),Contratação Jurídico Livre (Bi R$),Contratação Jurídico SFH (Bi R$),Direcionamento Aquisição (R$),Direcionamento Construção (R$),Direcionamento Aplicação (R$),Ativos Problemáticos - Home equity (R$),Ativos Problemáticos - SFH (R$),Ativos Problemáticos - FGTS (R$),Carteira de Crédito – Home equity (R$),Carteira de Crédito – Livre (R$),Carteira de Crédito - FGTS (R$),Carteira de Crédito - SFH (R$),Carteira de Crédito P.Jurídica - Comercial (R$),Carteira de Crédito P.Jurídica - Livre (R$),Carteira de Crédito P.Jurídica - FGTS (R$),Inadimplência - SFH (%),Inadimplência - FGTS (%),Inadimplência - Livre (%),Inadimplência - Comercial (%),Inadimplência - Home equity (%),Poupança e Empréstimo - SBPE (R$),Títulos - LH (R$),Títulos – LIG (R$),Títulos – CRI (R$),Títulos – LCI (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%)
0,-2.348480,-2.299843,-0.809794,-2.186463,-1.762689,0.489108,-0.377392,-2.320347,-1.997970,-1.329044,-0.409905,-1.240817,-1.095326,-0.230491,-0.194106,-1.022744,-0.900964,-1.382302,-0.536376,-1.414047,-1.088562,-1.252067,1.768350,-0.858773,-1.173480,-1.737283,-0.545087,-0.839415,1.416258,0.679015,1.062105,-1.190708,-0.026995,-1.615339,-0.300935,1.922051,-1.023713,-0.767541,2.723741,0.364808,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582
1,-2.305255,-2.261958,-0.812767,-2.149366,-1.742798,0.505628,-0.353105,-2.278259,-1.969272,-1.327215,-0.409905,-1.239136,-1.094441,-0.230491,-0.194106,-1.022744,-0.900964,-1.382302,-0.532013,-1.407886,-1.088562,-1.246382,1.768350,-0.858773,-1.173480,-1.737283,-0.545087,-0.839415,1.416258,0.674479,1.061740,-1.190708,-0.023784,-1.615339,-0.300935,1.922051,-1.023713,-0.762847,2.350880,0.368956,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356
2,-2.262030,-2.224072,-0.815740,-2.112268,-1.722906,0.522147,-0.328819,-2.236171,-1.940575,-1.326148,-0.409905,-1.237035,-1.093925,-0.230491,-0.194106,-1.022744,-0.900964,-1.382302,-0.529468,-1.404292,-1.088562,-1.243066,1.768350,-0.858773,-1.173480,-1.737283,-0.545087,-0.839415,1.416258,0.671833,1.061528,-1.190708,-0.021910,-1.615339,-0.300935,1.922051,-1.023713,-0.760108,2.123016,0.331110,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895
3,-2.218805,-2.186187,-0.818713,-2.075170,-1.703015,0.538666,-0.304532,-2.194083,-1.911877,-1.326033,-0.409905,-1.235858,-1.093869,-0.230491,-0.194106,-1.022744,-0.900964,-1.382302,-0.529195,-1.403907,-1.088562,-1.242711,1.768350,-0.858773,-1.173480,-1.737283,-0.545087,-0.839415,1.416258,0.671549,1.061505,-1.190708,-0.021709,-1.615339,-0.300935,1.922051,-1.023713,-0.759815,2.021477,0.244570,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628
4,-2.175580,-2.148302,-0.821686,-2.038072,-1.683123,0.555185,-0.280246,-2.151995,-1.883179,-1.319698,-0.406927,-1.233528,-1.088767,-0.227470,-0.190987,-1.022627,-0.901084,-1.381163,-0.527468,-1.398168,-1.088024,-1.238181,1.764433,-0.858957,-1.173107,-1.733705,-0.544159,-0.837772,1.411669,0.669072,1.056392,-1.190038,-0.022002,-1.613410,-0.298382,1.916280,-1.023685,-0.757341,1.887113,0.279013,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.135934,1.157797,1.241503,1.365742,1.057869,-1.253684,-1.748677,0.520808,1.453883,-0.124730,-1.726985,1.267258,-0.274933,-1.288888,-0.794961,2.073478,2.295585,1.588614,1.130581,1.248639,2.327796,0.899328,-2.088336,2.199956,1.570751,1.820906,-2.543863,-2.685027,-1.554847,-1.34

In [104]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      4.105
1      3.898
2      4.209
3      2.994
4      3.720
       ...  
236      NaN
237      NaN
238      NaN
239      NaN
240      NaN
Name: Acre - Consumo de Cimento (t), Length: 241, dtype: float64

In [105]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Acre - IDH Longevidade,Acre - IDH Renda,Acre - Desemprego,Acre - IDH,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado,Acre - IDH Educacao,Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),Contratação SFH (Bi R$),Contratação Jurídico Livre (Bi R$),Contratação Jurídico SFH (Bi R$),Direcionamento Aquisição (R$),Direcionamento Construção (R$),Direcionamento Aplicação (R$),Ativos Problemáticos - Home equity (R$),Ativos Problemáticos - SFH (R$),Ativos Problemáticos - FGTS (R$),Carteira de Crédito – Home equity (R$),Carteira de Crédito – Livre (R$),Carteira de Crédito - FGTS (R$),Carteira de Crédito - SFH (R$),Carteira de Crédito P.Jurídica - Comercial (R$),Carteira de Crédito P.Jurídica - Livre (R$),Carteira de Crédito P.Jurídica - FGTS (R$),Inadimplência - SFH (%),Inadimplência - FGTS (%),Inadimplência - Livre (%),Inadimplência - Comercial (%),Inadimplência - Home equity (%),Poupança e Empréstimo - SBPE (R$),Títulos - LH (R$),Títulos – LIG (R$),Títulos – CRI (R$),Títulos – LCI (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%)
0,-2.348480,-2.299843,-0.809794,-2.186463,-1.762689,0.489108,-0.377392,-2.320347,-1.997970,-1.329044,-0.409905,-1.240817,-1.095326,-0.230491,-0.194106,-1.022744,-0.900964,-1.382302,-0.536376,-1.414047,-1.088562,-1.252067,1.768350,-0.858773,-1.173480,-1.737283,-0.545087,-0.839415,1.416258,0.679015,1.062105,-1.190708,-0.026995,-1.615339,-0.300935,1.922051,-1.023713,-0.767541,2.723741,0.364808,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582
1,-2.305255,-2.261958,-0.812767,-2.149366,-1.742798,0.505628,-0.353105,-2.278259,-1.969272,-1.327215,-0.409905,-1.239136,-1.094441,-0.230491,-0.194106,-1.022744,-0.900964,-1.382302,-0.532013,-1.407886,-1.088562,-1.246382,1.768350,-0.858773,-1.173480,-1.737283,-0.545087,-0.839415,1.416258,0.674479,1.061740,-1.190708,-0.023784,-1.615339,-0.300935,1.922051,-1.023713,-0.762847,2.350880,0.368956,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356
2,-2.262030,-2.224072,-0.815740,-2.112268,-1.722906,0.522147,-0.328819,-2.236171,-1.940575,-1.326148,-0.409905,-1.237035,-1.093925,-0.230491,-0.194106,-1.022744,-0.900964,-1.382302,-0.529468,-1.404292,-1.088562,-1.243066,1.768350,-0.858773,-1.173480,-1.737283,-0.545087,-0.839415,1.416258,0.671833,1.061528,-1.190708,-0.021910,-1.615339,-0.300935,1.922051,-1.023713,-0.760108,2.123016,0.331110,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895
3,-2.218805,-2.186187,-0.818713,-2.075170,-1.703015,0.538666,-0.304532,-2.194083,-1.911877,-1.326033,-0.409905,-1.235858,-1.093869,-0.230491,-0.194106,-1.022744,-0.900964,-1.382302,-0.529195,-1.403907,-1.088562,-1.242711,1.768350,-0.858773,-1.173480,-1.737283,-0.545087,-0.839415,1.416258,0.671549,1.061505,-1.190708,-0.021709,-1.615339,-0.300935,1.922051,-1.023713,-0.759815,2.021477,0.244570,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628
4,-2.175580,-2.148302,-0.821686,-2.038072,-1.683123,0.555185,-0.280246,-2.151995,-1.883179,-1.319698,-0.406927,-1.233528,-1.088767,-0.227470,-0.190987,-1.022627,-0.901084,-1.381163,-0.527468,-1.398168,-1.088024,-1.238181,1.764433,-0.858957,-1.173107,-1.733705,-0.544159,-0.837772,1.411669,0.669072,1.056392,-1.190038,-0.022002,-1.613410,-0.298382,1.916280,-1.023685,-0.757341,1.887113,0.279013,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,1.455941,1.257015,1.455828,1.391708,1.134283,-1.537220,-1.492213,0.684545,1.354666,-0.972852,-1.329495,1.319037,-1.299201,-1.446097,-1.764215,1.130905,1.087724,1.096909,1.676523,1.449207,1.100435,1.355060,-0.394574,1.138603,1.278477,0.522836,-0.042735,0.586445,1.351864,-1.5901

In [106]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0       4.105
1       3.898
2       4.209
3       2.994
4       3.720
        ...  
157     9.081
158     9.545
159    10.130
160    14.451
161    13.218
Name: Acre - Consumo de Cimento (t), Length: 162, dtype: float64

In [107]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [108]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 46)

In [109]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,Acre - IDH Longevidade,Acre - IDH Renda,Acre - Desemprego,Acre - IDH,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado,Acre - IDH Educacao,Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),Contratação SFH (Bi R$),Contratação Jurídico Livre (Bi R$),Contratação Jurídico SFH (Bi R$),Direcionamento Aquisição (R$),Direcionamento Construção (R$),Direcionamento Aplicação (R$),Ativos Problemáticos - Home equity (R$),Ativos Problemáticos - SFH (R$),Ativos Problemáticos - FGTS (R$),Carteira de Crédito – Home equity (R$),Carteira de Crédito – Livre (R$),Carteira de Crédito - FGTS (R$),Carteira de Crédito - SFH (R$),Carteira de Crédito P.Jurídica - Comercial (R$),Carteira de Crédito P.Jurídica - Livre (R$),Carteira de Crédito P.Jurídica - FGTS (R$),Inadimplência - SFH (%),Inadimplência - FGTS (%),Inadimplência - Livre (%),Inadimplência - Comercial (%),Inadimplência - Home equity (%),Poupança e Empréstimo - SBPE (R$),Títulos - LH (R$),Títulos – LIG (R$),Títulos – CRI (R$),Títulos – LCI (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%)
126,0.398190,0.790414,0.818598,0.403592,0.895819,-0.443620,0.224228,1.241058,0.474766,0.561200,0.469010,0.654287,0.514061,0.169137,-0.058348,0.381693,0.263992,0.600448,0.384917,0.576969,0.265242,0.888320,-0.202741,0.191660,0.602105,0.155824,1.350509,1.172681,-0.264671,-0.194824,-0.162207,0.453853,0.188388,0.773189,0.592023,-0.689873,0.403484,0.557912,0.888984,-0.230612,-0.460555,-1.131704,-0.368821,0.651397,0.944085,0.045243
127,0.398778,0.772820,0.846240,0.408649,0.913558,-0.464307,0.187909,1.244382,0.485404,0.487970,0.414665,0.688711,0.442312,0.103544,-0.133421,0.412779,0.295413,0.619650,0.447173,0.598465,0.285110,0.933789,-0.155709,0.221973,0.636016,0.111878,1.373026,1.179922,-0.189339,-0.240395,-0.071411,0.479039,0.262788,0.782160,0.550513,-0.689873,0.436846,0.616673,0.954254,-0.156322,-0.440372,-0.613483,-0.328087,0.664707,0.947319,0.061828
128,0.399367,0.755226,0.873883,0.413707,0.931297,-0.484993,0.151590,1.247706,0.496042,0.411684,0.358242,0.723533,0.367653,0.035698,-0.210949,0.444315,0.327387,0.638700,0.511105,0.619935,0.305090,0.979938,-0.106412,0.252850,0.670325,0.065792,1.395529,1.186617,-0.110946,-0.287128,0.022759,0.504395,0.339514,0.790494,0.507329,-0.689873,0.470723,0.676845,1.045217,-0.120008,-0.419247,-0.542919,-0.176031,0.678017,0.950553,0.046225
129,0.399956,0.737631,0.901525,0.418765,0.949036,-0.505680,0.115270,1.251030,0.506679,0.332303,0.299713,0.758754,0.290047,-0.034429,-0.290961,0.476306,0.359921,0.657591,0.576734,0.641378,0.325180,1.026772,-0.054817,0.284295,0.705035,0.017540,1.418016,1.192754,-0.029453,-0.335037,0.120347,0.529920,0.418596,0.798179,0.462450,-0.689873,0.505118,0.738444,1.176395,-0.143194,-0.397019,-1.334517,-0.113037,0.691327,0.953786,0.032522
130,0.400545,0.720037,0.929167,0.423822,0.966775,-0.526367,0.078951,1.254354,0.517317,0.249788,0.239053,0.794376,0.209457,-0.106863,-0.373485,0.508754,0.393019,0.676320,0.644076,0.662791,0.345382,1.074297,-0.000896,0.316313,0.740146,-0.032908,1.440483,1.198325,0.055179,-0.384135,0.221394,0.555614,0.500061,0.805204,0.415855,-0.689873,0.540037,0.801485,1.303259,-0.028712,-0.376532,-1.511762,-0.022703,0.704637,0.957020,0.042757
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.135934,1.157797,1.241503,1.365742,1.057869,-1.253684,-1.748677,0.520808,1.453883,-0.124730,-1.726985,1.267258,-0.274933,-1.288888,-0.794961,2.073478,2.295585,1.588614,1.130581,1.248639,2.327796,0.899328,-2.088336,2.199956,1.570751,1.820906,-2.543863,-2.685027,-1.554847,-1.343430,0.314933,1.067345,0.754981,1.520670,-2.253017,1.977516,1.711283,1.052051,-2.010387,2.953658,-0.572934

In [110]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 46)

In [111]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [112]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(360, activation='tanh', 
                             return_sequences=True, 
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(180, return_sequences=True, activation='tanh'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.LSTM(36, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [113]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(50)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [114]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[1395137673, 1797660912, 1789249561, 160154757, 2929638250, 3960621930, 3708940670, 1941888725, 2781846417, 2790921221, 4077626240, 3136687791, 2980046571, 1101180908, 4146756185, 96401899, 1367152643, 1353497403, 4167050643, 1683411650, 3602131458, 2620864525, 4024216184, 566197045, 2976542477, 3795237636, 2276002079, 455765851, 3706714495, 4284805438, 1035829787, 491247132, 817584104, 1789399966, 2625022617, 4107620572, 1282935621, 4047531683, 251132367, 517178804, 1704470915, 2871130671, 3927360690, 1605032854, 652481715, 3819506268, 3719800673, 2063403957, 712685968, 74354084]


Step: 0 ___________________________________________
val_loss: 2.2702479362487793
winner_seed: 1395137673


Step: 1 ___________________________________________
val_loss: 2.0406484603881836
winner_seed: 1797660912


Step: 2 ___________________________________________
val_loss: 4.354349136352539


Step: 3 ___________________________________________
val_loss: 5.730213165283203


Step: 4 ________________________

In [ ]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 2s 160ms/step - loss: 157.8092 - val_loss: 61.2155
Epoch 2/10000
4/4 [==============================] - 0s 11ms/step - loss: 48.4413 - val_loss: 12.7371
Epoch 3/10000
4/4 [==============================] - 0s 10ms/step - loss: 15.4225 - val_loss: 9.6380
Epoch 4/10000
4/4 [==============================] - 0s 11ms/step - loss: 11.5078 - val_loss: 10.1569
Epoch 5/10000
4/4 [==============================] - 0s 10ms/step - loss: 11.0049 - val_loss: 6.1770
Epoch 6/10000
4/4 [==============================] - 0s 10ms/step - loss: 9.6341 - val_loss: 5.6742
Epoch 7/10000
4/4 [==============================] - 0s 11ms/step - loss: 8.9013 - val_loss: 10.6263
Epoch 8/10000
4/4 [==============================] - 0s 10ms/step - loss: 9.2992 - val_loss: 6.7488
Epoch 9/10000
4/4 [==============================] - 0s 11ms/step - loss: 8.8298 - val_loss: 12.6400
Epoch 10/10000
4/4 [==============================] - 0s 10ms/step - loss: 9.0406 - val_

4/4 [==============================] - 0s 10ms/step - loss: 6.6720 - val_loss: 4.9132
Epoch 83/10000
4/4 [==============================] - 0s 10ms/step - loss: 8.2837 - val_loss: 5.8105
Epoch 84/10000
4/4 [==============================] - 0s 10ms/step - loss: 6.6585 - val_loss: 6.4477
Epoch 85/10000
4/4 [==============================] - 0s 10ms/step - loss: 6.4467 - val_loss: 5.1442
Epoch 86/10000
4/4 [==============================] - 0s 10ms/step - loss: 6.8489 - val_loss: 5.4284
Epoch 87/10000
4/4 [==============================] - 0s 10ms/step - loss: 6.6845 - val_loss: 6.4809
Epoch 88/10000
4/4 [==============================] - 0s 10ms/step - loss: 6.6837 - val_loss: 5.4422
Epoch 89/10000
4/4 [==============================] - 0s 10ms/step - loss: 6.5193 - val_loss: 5.7242
Epoch 90/10000
4/4 [==============================] - 0s 10ms/step - loss: 6.2923 - val_loss: 5.9665
Epoch 91/10000
4/4 [==============================] - 0s 10ms/step - loss: 6.6554 - val_loss: 6.5659
Epoch

4/4 [==============================] - 0s 10ms/step - loss: 6.3446 - val_loss: 6.5065
Epoch 163/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.9980 - val_loss: 8.7805
Epoch 164/10000
4/4 [==============================] - 0s 10ms/step - loss: 7.3455 - val_loss: 6.2024
Epoch 165/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.7807 - val_loss: 6.6577
Epoch 166/10000
4/4 [==============================] - 0s 11ms/step - loss: 6.0096 - val_loss: 7.4399
Epoch 167/10000
4/4 [==============================] - 0s 11ms/step - loss: 6.0841 - val_loss: 7.2298
Epoch 168/10000
4/4 [==============================] - 0s 11ms/step - loss: 6.3945 - val_loss: 8.0281
Epoch 169/10000
4/4 [==============================] - 0s 11ms/step - loss: 6.1050 - val_loss: 6.2499
Epoch 170/10000
4/4 [==============================] - 0s 11ms/step - loss: 6.2513 - val_loss: 6.8303
Epoch 171/10000
4/4 [==============================] - 0s 10ms/step - loss: 6.0943 - val_loss: 7.2

4/4 [==============================] - 0s 10ms/step - loss: 5.8202 - val_loss: 5.5880
Epoch 243/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.7898 - val_loss: 6.4759
Epoch 244/10000
4/4 [==============================] - 0s 11ms/step - loss: 5.9375 - val_loss: 5.7792
Epoch 245/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.6549 - val_loss: 7.0905
Epoch 246/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.6377 - val_loss: 7.1532
Epoch 247/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.9658 - val_loss: 6.2094
Epoch 248/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.5257 - val_loss: 7.0554
Epoch 249/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.8715 - val_loss: 7.0042
Epoch 250/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.7049 - val_loss: 7.5438
Epoch 251/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.9003 - val_loss: 6.1

4/4 [==============================] - 0s 10ms/step - loss: 5.4788 - val_loss: 5.8309
Epoch 323/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.5462 - val_loss: 5.1629
Epoch 324/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.6985 - val_loss: 5.3050
Epoch 325/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.5792 - val_loss: 5.6183
Epoch 326/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.8274 - val_loss: 7.4563
Epoch 327/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.5973 - val_loss: 7.6344
Epoch 328/10000
4/4 [==============================] - 0s 10ms/step - loss: 6.0885 - val_loss: 8.0817
Epoch 329/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.6455 - val_loss: 6.5004
Epoch 330/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.7594 - val_loss: 7.8788
Epoch 331/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.9531 - val_loss: 5.6

...

4/4 [==============================] - 0s 10ms/step - loss: 5.1288 - val_loss: 5.2355
Epoch 883/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.9142 - val_loss: 6.7086
Epoch 884/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.1074 - val_loss: 5.3199
Epoch 885/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.3786 - val_loss: 4.9953
Epoch 886/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.7929 - val_loss: 5.6706
Epoch 887/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.8087 - val_loss: 7.3163
Epoch 888/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.3134 - val_loss: 5.7761
Epoch 889/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.9455 - val_loss: 6.1574
Epoch 890/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.0841 - val_loss: 5.1343
Epoch 891/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.6362 - val_loss: 5.8

4/4 [==============================] - 0s 10ms/step - loss: 3.9151 - val_loss: 5.4663
Epoch 963/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.1670 - val_loss: 5.3326
Epoch 964/10000
4/4 [==============================] - 0s 10ms/step - loss: 3.9556 - val_loss: 5.3584
Epoch 965/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.0201 - val_loss: 6.3363
Epoch 966/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.0914 - val_loss: 5.0596
Epoch 967/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.4055 - val_loss: 5.0708
Epoch 968/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.2979 - val_loss: 5.9731
Epoch 969/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.4557 - val_loss: 4.8519
Epoch 970/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.3315 - val_loss: 4.8547
Epoch 971/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.4511 - val_loss: 6.9

4/4 [==============================] - 0s 11ms/step - loss: 3.1839 - val_loss: 4.8549
Epoch 1043/10000
4/4 [==============================] - 0s 11ms/step - loss: 3.5986 - val_loss: 6.5869
Epoch 1044/10000
4/4 [==============================] - 0s 11ms/step - loss: 3.0839 - val_loss: 4.6443
Epoch 1045/10000
4/4 [==============================] - 0s 11ms/step - loss: 3.4548 - val_loss: 6.0764
Epoch 1046/10000
4/4 [==============================] - 0s 11ms/step - loss: 3.6980 - val_loss: 7.4290
Epoch 1047/10000
4/4 [==============================] - 0s 11ms/step - loss: 3.3608 - val_loss: 4.7360
Epoch 1048/10000
4/4 [==============================] - 0s 10ms/step - loss: 3.9332 - val_loss: 5.8609
Epoch 1049/10000
4/4 [==============================] - 0s 10ms/step - loss: 3.6558 - val_loss: 4.8264
Epoch 1050/10000
4/4 [==============================] - 0s 10ms/step - loss: 3.6331 - val_loss: 5.0246
Epoch 1051/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.7223 - val_

4/4 [==============================] - 0s 10ms/step - loss: 4.0573 - val_loss: 4.6027
Epoch 1122/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.4463 - val_loss: 6.4350
Epoch 1123/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.3624 - val_loss: 4.9150
Epoch 1124/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.0007 - val_loss: 4.8032
Epoch 1125/10000
4/4 [==============================] - 0s 10ms/step - loss: 3.9378 - val_loss: 4.0384
Epoch 1126/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.0000 - val_loss: 5.9778
Epoch 1127/10000
4/4 [==============================] - 0s 10ms/step - loss: 3.7647 - val_loss: 5.0168
Epoch 1128/10000
4/4 [==============================] - 0s 10ms/step - loss: 3.6120 - val_loss: 4.1787
Epoch 1129/10000
4/4 [==============================] - 0s 10ms/step - loss: 3.5936 - val_loss: 5.9560
Epoch 1130/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.2987 - val_

4/4 [==============================] - 0s 10ms/step - loss: 3.1476 - val_loss: 5.7611
Epoch 1201/10000
4/4 [==============================] - 0s 10ms/step - loss: 3.3110 - val_loss: 5.1108
Epoch 1202/10000
4/4 [==============================] - 0s 10ms/step - loss: 3.1257 - val_loss: 3.5716
Epoch 1203/10000
4/4 [==============================] - 0s 10ms/step - loss: 2.8374 - val_loss: 6.3745
Epoch 1204/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.1012 - val_loss: 4.9426
Epoch 1205/10000
4/4 [==============================] - 0s 10ms/step - loss: 3.2118 - val_loss: 4.1499
Epoch 1206/10000
4/4 [==============================] - 0s 10ms/step - loss: 2.9463 - val_loss: 4.1044
Epoch 1207/10000
4/4 [==============================] - 0s 10ms/step - loss: 3.0294 - val_loss: 5.2977
Epoch 1208/10000
4/4 [==============================] - 0s 10ms/step - loss: 2.8787 - val_loss: 4.3657
Epoch 1209/10000
4/4 [==============================] - 0s 10ms/step - loss: 3.4761 - val_

In [ ]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [ ]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

In [ ]:
display(mae)
display(mape)

In [ ]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [ ]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             test_input, 
                                             test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)